In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
#groups: Οι πολιτικές ομάδες του Ευρωκοινοβουλίου
groups1984 = pd.read_csv('groups_84.csv', sep=';')
groups1984 = groups1984[groups1984.LANGUAGE_ID.str.contains('EL')]
groups1984 = groups1984.rename(columns={'ID':'GROUP_ID'})
groups1984

,GROUP_ID,LANGUAGE_ID,ACRONYM,LABEL
6,GP0201,EL,S,Σοσιαλιστική Ομάδα
30,GP0202,EL,PPE,Ομάδα του Ευρωπαϊκού Λαϊκού Κόμματος (Χριστιαν...
54,GP0203,EL,ED,Ομάδα των Ευρωπαίων Δημοκρατών
78,GP0204,EL,COM,Κομμουνιστική ομάδα και Προσκείμενοι
102,GP0205,EL,L,Φιλελεύθερη και Δημοκρατική Ομάδα
126,GP0206,EL,RDE,Ομάδα Συνασπισμού των Ευρωπαίων Δημοκρατών
150,GP0207,EL,ARC,Ομάδα 'Ουράνιο Τόξο': Ομοσπονδία του Ευρωπαϊκο...
174,GP0208,EL,DR,Ομάδα των Κομμάτων της Ευρωπαϊκής Δεξιάς
198,NaN,EL,NI,Μη Εγγεγραμμένοι


In [4]:
#parties: Τα εθνικά κόμματα μέσα στο Ευρωκοινοβούλιο
parties1984 = pd.read_csv('parties_84.csv', sep=';')
parties1984 = parties1984[['DIVISION_ID','ID','ACRONYM','LABEL']].copy()
parties1984 = parties1984[parties1984.DIVISION_ID.str.contains('EL')]
parties1984 = parties1984.rename(columns={'ID':'PARTY_ID'})
parties1984

,DIVISION_ID,PARTY_ID,ACRONYM,LABEL
37,EL,EL01,PASOK,Panellinio Sosialistiko Kinima
38,EL,EL02,ND,Nea Dhimokratia
39,EL,EL03,KKE,Kommunistiko Komma Elladas
40,EL,EL04,E.P.EN,Ethniki Politiki Enosis
41,EL,EL90,Other parties,Other parties


In [5]:
#Αποτελέσματα ανά πολιτική ομάδα για την Ελλάδα
omada1984 = pd.read_csv('el_84.csv', sep=';')
omada1984 = omada1984[['GROUP_ID','SEATS_TOTAL','SEATS_PERCENT_EU']].copy()
omada1984

,GROUP_ID,SEATS_TOTAL,SEATS_PERCENT_EU
0,GP0201,10,2.30
1,GP0202,9,2.07
2,GP0203,0,0.00
3,GP0204,4,0.92
4,GP0205,0,0.00
5,GP0206,0,0.00
6,GP0207,0,0.00
7,GP0208,1,0.23
8,NaN,0,0.00


In [8]:
#Κατάταξη ανά εθνικά κόμματα και πολιτικές ομάδες, δηλαδή πόσες ΄έδρες έχει το κάθε ελληνικό κόμμα στην αντίστοιχη πολιτική 
#ομάδα
katataksh1984 = pd.read_csv('katataksh_84.csv', sep=';')
katataksh1984 = katataksh1984[['PARTY_ID','SEATS_TOTAL','GROUP_ID']].copy()
katataksh1984

,PARTY_ID,SEATS_TOTAL,GROUP_ID
0,EL01,10,GP0201
1,EL02,9,GP0202
2,EL03,4,GP0204
3,EL04,1,GP0208


In [9]:
#Αποτελέσματα ανά πολιτική ομάδα για την Ευρώπη
eu1984 = pd.read_csv('eu_84.csv', sep=';')
eu1984 = eu1984[['GROUP_ID', 'SEATS_TOTAL','SEATS_PERCENT_EU']]
eu1984

,GROUP_ID,SEATS_TOTAL,SEATS_PERCENT_EU
0,GP0201,130,29.95
1,GP0202,110,25.35
2,GP0203,50,11.52
3,GP0204,41,9.45
4,GP0205,31,7.14
5,GP0206,29,6.68
6,GP0207,20,4.61
7,GP0208,16,3.69
8,NaN,7,1.61


In [10]:
#Ξεκινάμε να εν΄΄ωνουμε τα dataframe μας με βάση κοινές στήλες
#Πρώτα με το GROUP_ID, που είναι οι πολιτικές ομάδες του Ευρωκοινοβουλίου
df1984 = pd.merge(katataksh1984, omada1984, on='GROUP_ID', how='outer')
df1984

,PARTY_ID,SEATS_TOTAL_x,GROUP_ID,SEATS_TOTAL_y,SEATS_PERCENT_EU
0,EL01,10.0,GP0201,10,2.30
1,EL02,9.0,GP0202,9,2.07
2,EL03,4.0,GP0204,4,0.92
3,EL04,1.0,GP0208,1,0.23
4,NaN,NaN,GP0203,0,0.00
5,NaN,NaN,GP0205,0,0.00
6,NaN,NaN,GP0206,0,0.00
7,NaN,NaN,GP0207,0,0.00
8,NaN,NaN,NaN,0,0.00


In [11]:
df1984 = pd.merge(df1984, groups1984, on='GROUP_ID', how='outer')
df1984

,PARTY_ID,SEATS_TOTAL_x,GROUP_ID,SEATS_TOTAL_y,SEATS_PERCENT_EU,LANGUAGE_ID,ACRONYM,LABEL
0,EL01,10.0,GP0201,10,2.30,EL,S,Σοσιαλιστική Ομάδα
1,EL02,9.0,GP0202,9,2.07,EL,PPE,Ομάδα του Ευρωπαϊκού Λαϊκού Κόμματος (Χριστιαν...
2,EL03,4.0,GP0204,4,0.92,EL,COM,Κομμουνιστική ομάδα και Προσκείμενοι
3,EL04,1.0,GP0208,1,0.23,EL,DR,Ομάδα των Κομμάτων της Ευρωπαϊκής Δεξιάς
4,NaN,NaN,GP0203,0,0.00,EL,ED,Ομάδα των Ευρωπαίων Δημοκρατών
5,NaN,NaN,GP0205,0,0.00,EL,L,Φιλελεύθερη και Δημοκρατική Ομάδα
6,NaN,NaN,GP0206,0,0.00,EL,RDE,Ομάδα Συνασπισμού των Ευρωπαίων Δημοκρατών
7,NaN,NaN,GP0207,0,0.00,EL,ARC,Ομάδα 'Ουράνιο Τόξο': Ομοσπονδία του Ευρωπαϊκο...
8,NaN,NaN,NaN,0,0.00,EL,NI,Μη Εγγεγραμμένοι


In [12]:
#Στη συνέχεια με το PARTY_ID, που είναι τα εθνικά κόμματα της Ελλάδας
df1984 = pd.merge(df1984, parties1984, on='PARTY_ID', how='outer')
df1984

,PARTY_ID,SEATS_TOTAL_x,GROUP_ID,SEATS_TOTAL_y,SEATS_PERCENT_EU,LANGUAGE_ID,ACRONYM_x,LABEL_x,DIVISION_ID,ACRONYM_y,LABEL_y
0,EL01,10.0,GP0201,10.0,2.30,EL,S,Σοσιαλιστική Ομάδα,EL,PASOK,Panellinio Sosialistiko Kinima
1,EL02,9.0,GP0202,9.0,2.07,EL,PPE,Ομάδα του Ευρωπαϊκού Λαϊκού Κόμματος (Χριστιαν...,EL,ND,Nea Dhimokratia
2,EL03,4.0,GP0204,4.0,0.92,EL,COM,Κομμουνιστική ομάδα και Προσκείμενοι,EL,KKE,Kommunistiko Komma Elladas
3,EL04,1.0,GP0208,1.0,0.23,EL,DR,Ομάδα των Κομμάτων της Ευρωπαϊκής Δεξιάς,EL,E.P.EN,Ethniki Politiki Enosis
4,NaN,NaN,GP0203,0.0,0.00,EL,ED,Ομάδα των Ευρωπαίων Δημοκρατών,NaN,NaN,NaN
5,NaN,NaN,GP0205,0.0,0.00,EL,L,Φιλελεύθερη και Δημοκρατική Ομάδα,NaN,NaN,NaN
6,NaN,NaN,GP0206,0.0,0.00,EL,RDE,Ομάδα Συνασπισμού των Ευρωπαίων Δημοκρατών,NaN,NaN,NaN
7,NaN,NaN,GP0207,0.0,0.00,EL,ARC,Ομάδα 'Ουράνιο Τόξο': Ομοσπονδία του Ευρωπαϊκο...,NaN,NaN,NaN
8,NaN,NaN,NaN,0.0,0.00,EL,NI,Μη Εγγεγραμμένοι,NaN,NaN,NaN
9,EL90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EL,Other parties,Other parties


In [14]:
#Στη συνέχεια μετονομάζουμε κάποιες στήλες ώστε να ξεχωρίζουμε τις θέσεις των εθνικών κομμάτων (SEATS_TOTAL_P από το Party)
#και τις θέσεις των πολιτικών ομάδων (SEATS_TOTAL_G από το Groups)
df1984 = df1984[['PARTY_ID','SEATS_TOTAL_x','GROUP_ID','SEATS_TOTAL_y','SEATS_PERCENT_EU','LABEL_x', 'ACRONYM_y']].copy()
df1984 = df1984.rename(columns={'SEATS_TOTAL_x':'SEATS_TOTAL_P'})
df1984 = df1984.rename(columns={'SEATS_TOTAL_y':'SEATS_TOTAL_G'})
df1984['YEAR'] = 1984
df1984

,PARTY_ID,SEATS_TOTAL_P,GROUP_ID,SEATS_TOTAL_G,SEATS_PERCENT_EU,LABEL_x,ACRONYM_y,YEAR
0,EL01,10.0,GP0201,10.0,2.30,Σοσιαλιστική Ομάδα,PASOK,1984
1,EL02,9.0,GP0202,9.0,2.07,Ομάδα του Ευρωπαϊκού Λαϊκού Κόμματος (Χριστιαν...,ND,1984
2,EL03,4.0,GP0204,4.0,0.92,Κομμουνιστική ομάδα και Προσκείμενοι,KKE,1984
3,EL04,1.0,GP0208,1.0,0.23,Ομάδα των Κομμάτων της Ευρωπαϊκής Δεξιάς,E.P.EN,1984
4,NaN,NaN,GP0203,0.0,0.00,Ομάδα των Ευρωπαίων Δημοκρατών,NaN,1984
5,NaN,NaN,GP0205,0.0,0.00,Φιλελεύθερη και Δημοκρατική Ομάδα,NaN,1984
6,NaN,NaN,GP0206,0.0,0.00,Ομάδα Συνασπισμού των Ευρωπαίων Δημοκρατών,NaN,1984
7,NaN,NaN,GP0207,0.0,0.00,Ομάδα 'Ουράνιο Τόξο': Ομοσπονδία του Ευρωπαϊκο...,NaN,1984
8,NaN,NaN,NaN,0.0,0.00,Μη Εγγεγραμμένοι,NaN,1984
9,EL90,NaN,NaN,NaN,NaN,NaN,Other parties,1984


In [15]:
df1984 = pd.merge(df1984, eu1984, on='GROUP_ID', how='outer')
df1984

,PARTY_ID,SEATS_TOTAL_P,GROUP_ID,SEATS_TOTAL_G,SEATS_PERCENT_EU_x,LABEL_x,ACRONYM_y,YEAR,SEATS_TOTAL,SEATS_PERCENT_EU_y
0,EL01,10.0,GP0201,10.0,2.30,Σοσιαλιστική Ομάδα,PASOK,1984,130,29.95
1,EL02,9.0,GP0202,9.0,2.07,Ομάδα του Ευρωπαϊκού Λαϊκού Κόμματος (Χριστιαν...,ND,1984,110,25.35
2,EL03,4.0,GP0204,4.0,0.92,Κομμουνιστική ομάδα και Προσκείμενοι,KKE,1984,41,9.45
3,EL04,1.0,GP0208,1.0,0.23,Ομάδα των Κομμάτων της Ευρωπαϊκής Δεξιάς,E.P.EN,1984,16,3.69
4,NaN,NaN,GP0203,0.0,0.00,Ομάδα των Ευρωπαίων Δημοκρατών,NaN,1984,50,11.52
5,NaN,NaN,GP0205,0.0,0.00,Φιλελεύθερη και Δημοκρατική Ομάδα,NaN,1984,31,7.14
6,NaN,NaN,GP0206,0.0,0.00,Ομάδα Συνασπισμού των Ευρωπαίων Δημοκρατών,NaN,1984,29,6.68
7,NaN,NaN,GP0207,0.0,0.00,Ομάδα 'Ουράνιο Τόξο': Ομοσπονδία του Ευρωπαϊκο...,NaN,1984,20,4.61
8,NaN,NaN,NaN,0.0,0.00,Μη Εγγεγραμμένοι,NaN,1984,7,1.61
9,EL90,NaN,NaN,NaN,NaN,NaN,Other parties,1984,7,1.61


In [16]:
groups1989 = pd.read_csv('groups_89.csv', sep=';')
groups1989 = groups1989[groups1989.LANGUAGE_ID.str.contains('EL')]
groups1989 = groups1989.rename(columns={'ID':'GROUP_ID'})

In [17]:
parties1989 = pd.read_csv('parties_89.csv', sep=';')
parties1989 = parties1989[['DIVISION_ID','ID','ACRONYM']].copy()
parties1989 = parties1989[parties1989.DIVISION_ID.str.contains('EL')]
parties1989 = parties1989.rename(columns={'ID':'PARTY_ID'})

In [18]:
omada1989 = pd.read_csv('el_89.csv', sep=';')
omada1989 = omada1989[['GROUP_ID','SEATS_TOTAL','SEATS_PERCENT_EU']].copy()

In [19]:
eu1989 = pd.read_csv('eu_89.csv', sep=';')
eu1989 = eu1989[['GROUP_ID', 'SEATS_TOTAL','SEATS_PERCENT_EU']]

In [20]:
katataksh1989 = pd.read_csv('katataksh_89.csv', sep=';')
katataksh1989 = katataksh1989[['PARTY_ID','SEATS_TOTAL','GROUP_ID']].copy()

In [21]:
df1989 = pd.merge(katataksh1989, omada1989, on='GROUP_ID', how='outer')
df1989 = pd.merge(df1989, groups1989, on='GROUP_ID', how='outer')
df1989 = pd.merge(df1989, parties1989, on='PARTY_ID', how='outer')

In [22]:
df1989 = df1989[['PARTY_ID','SEATS_TOTAL_x','GROUP_ID','SEATS_TOTAL_y','SEATS_PERCENT_EU','LABEL', 'ACRONYM_y']].copy()
df1989 = df1989.rename(columns={'SEATS_TOTAL_x':'SEATS_TOTAL_P'})
df1989 = df1989.rename(columns={'SEATS_TOTAL_y':'SEATS_TOTAL_G'})
df1989['YEAR'] = 1989

In [23]:
df1989 = pd.merge(df1989, eu1989, on='GROUP_ID', how='outer')
df1989

,PARTY_ID,SEATS_TOTAL_P,GROUP_ID,SEATS_TOTAL_G,SEATS_PERCENT_EU_x,LABEL,ACRONYM_y,YEAR,SEATS_TOTAL,SEATS_PERCENT_EU_y
0,EL01,9.0,GP0401,9.0,1.74,Σοσιαλιστική Ομάδα,PA.SO.K.,1989,180,34.75
1,EL02,10.0,GP0402,10.0,1.93,Ομάδα του Ευρωπαϊκού Λαϊκού Κόμματος (Χριστιαν...,N.D.,1989,121,23.36
2,EL03,1.0,GP0406,1.0,0.19,Ομάδα για την Ευρωπαϊκή Ενωτική Αριστερά,SYN,1989,28,5.41
3,EL03,3.0,GP0409,3.0,0.58,Συνασπισμός της Αριστεράς,SYN,1989,14,2.70
4,EL04,1.0,GP0407,1.0,0.19,Ομάδα Συνασπισμού των Ευρωπαίων Δημοκρατών,DI.ANA,1989,20,3.86
5,NaN,NaN,GP0403,0.0,0.00,"Φιλελεύθερη, Δημοκρατική και Μεταρρυθμιστική Ο...",NaN,1989,49,9.46
6,NaN,NaN,GP0404,0.0,0.00,Ομάδα των Ευρωπαίων Δημοκρατών,NaN,1989,34,6.56
7,NaN,NaN,GP0405,0.0,0.00,Ομάδα των Πρασίνων στο Ευρωπαϊκό Κοινοβούλιο,NaN,1989,30,5.79
8,NaN,NaN,GP0408,0.0,0.00,Ομάδα Τεχνικού Συντονισμού των Κομμάτων της Ευ...,NaN,1989,17,3.28
9,NaN,NaN,GP0410,0.0,0.00,Ομάδα Ουράνιου Τόξου στο Ευρωπαϊκό Κοινοβούλιο,NaN,1989,13,2.51


In [24]:
groups1994 = pd.read_csv('groups_94.csv', sep=';')
groups1994 = groups1994[groups1994.LANGUAGE_ID.str.contains('EL')]
groups1994 = groups1994.rename(columns={'ID':'GROUP_ID'})

In [25]:
parties1994 = pd.read_csv('parties_94.csv', sep=';')
parties1994 = parties1994[['DIVISION_ID','ID','ACRONYM']].copy()
parties1994 = parties1994[parties1994.DIVISION_ID.str.contains('EL')]
parties1994 = parties1994.rename(columns={'ID':'PARTY_ID'})

In [26]:
omada1994 = pd.read_csv('el_94.csv', sep=';')
omada1994 = omada1994[['GROUP_ID','SEATS_TOTAL','SEATS_PERCENT_EU']].copy()

In [27]:
eu1994 = pd.read_csv('eu_94.csv', sep=';')
eu1994 = eu1994[['GROUP_ID', 'SEATS_TOTAL','SEATS_PERCENT_EU']]

In [28]:
katataksh1994 = pd.read_csv('katataksh_94.csv', sep=';')
katataksh1994 = katataksh1994[['PARTY_ID','SEATS_TOTAL','GROUP_ID']].copy()

In [29]:
df1994 = pd.merge(katataksh1994, omada1994, on='GROUP_ID', how='outer')
df1994 = pd.merge(df1994, groups1994, on='GROUP_ID', how='outer')
df1994 = pd.merge(df1994, parties1994, on='PARTY_ID', how='outer')
df1994 = df1994[['PARTY_ID','SEATS_TOTAL_x','GROUP_ID','SEATS_TOTAL_y','SEATS_PERCENT_EU','LABEL', 'ACRONYM_y']].copy()
df1994 = df1994.rename(columns={'SEATS_TOTAL_x':'SEATS_TOTAL_P'})
df1994 = df1994.rename(columns={'SEATS_TOTAL_y':'SEATS_TOTAL_G'})
df1994['YEAR'] = 1994
df1994 = pd.merge(df1994, eu1994, on='GROUP_ID', how='outer')
df1994

,PARTY_ID,SEATS_TOTAL_P,GROUP_ID,SEATS_TOTAL_G,SEATS_PERCENT_EU_x,LABEL,ACRONYM_y,YEAR,SEATS_TOTAL,SEATS_PERCENT_EU_y
0,EL01,10.0,GP0601,10.0,1.76,Ομάδα του Ευρωπαϊκού Σοσιαλιστικού Κόμματος,PASOK,1994,198,34.92
1,EL02,9.0,GP0602,9.0,1.59,Ομάδα του Ευρωπαϊκού Λαϊκού Κόμματος (Χριστιαν...,N.D.,1994,156,27.51
2,EL03,2.0,GP0604,4.0,0.71,Συνομοσπονδιακή Ομάδα της Ευρωπαϊκής Ενωτικής ...,KKE,1994,28,4.94
3,EL04,2.0,GP0604,4.0,0.71,Συνομοσπονδιακή Ομάδα της Ευρωπαϊκής Ενωτικής ...,SYN,1994,28,4.94
4,EL05,2.0,GP0606,2.0,0.35,Ομάδα Συνασπισμού των Ευρωπαίων Δημοκρατών,POL.AN,1994,26,4.59
5,NaN,NaN,GP0603,0.0,0.00,Ομάδα του Ευρωπαϊκού Κόμματος των Φιλελευθέρων...,NaN,1994,44,7.76
6,NaN,NaN,GP0605,0.0,0.00,Ομάδα Forza Europa,NaN,1994,27,4.76
7,NaN,NaN,GP0607,0.0,0.00,Ομάδα των Πρασίνων στο Ευρωπαϊκό Κοινοβούλιο,NaN,1994,23,4.06
8,NaN,NaN,GP0608,0.0,0.00,Ομάδα της Ευρωπαϊκής Ριζοσπαστικής Συμμαχίας,NaN,1994,19,3.35
9,NaN,NaN,GP0609,0.0,0.00,Ομάδα Ευρώπη των Εθνών (Ομάδα συντονισμού),NaN,1994,19,3.35


In [30]:
groups1999 = pd.read_csv('groups_99.csv', sep=';')
groups1999 = groups1999[groups1999.LANGUAGE_ID.str.contains('EL')]
groups1999 = groups1999.rename(columns={'ID':'GROUP_ID'})

In [31]:
parties1999 = pd.read_csv('parties_99.csv', sep=';')
parties1999 = parties1999[['DIVISION_ID','ID','ACRONYM']].copy()
parties1999 = parties1999[parties1999.DIVISION_ID.str.contains('EL')]
parties1999 = parties1999.rename(columns={'ID':'PARTY_ID'})

In [32]:
omada1999 = pd.read_csv('el_99.csv', sep=';')
omada1999 = omada1999[['GROUP_ID','SEATS_TOTAL','SEATS_PERCENT_EU']].copy()

In [33]:
eu1999 = pd.read_csv('eu_99.csv', sep=';')
eu1999 = eu1999[['GROUP_ID', 'SEATS_TOTAL','SEATS_PERCENT_EU']]

In [34]:
katataksh1999 = pd.read_csv('katataksh_99.csv', sep=';')
katataksh1999 = katataksh1999[['PARTY_ID','SEATS_TOTAL','GROUP_ID']].copy()

In [35]:
df1999 = pd.merge(katataksh1999, omada1999, on='GROUP_ID', how='outer')
df1999 = pd.merge(df1999, groups1999, on='GROUP_ID', how='outer')
df1999 = pd.merge(df1999, parties1999, on='PARTY_ID', how='outer')
df1999 = df1999[['PARTY_ID','SEATS_TOTAL_x','GROUP_ID','SEATS_TOTAL_y','SEATS_PERCENT_EU','LABEL', 'ACRONYM_y']].copy()
#Ta SEATS_TOTAL_P αντιστοιχούν στις θέσεις των εθνικών κομμάτων και τα SEATS_TOTAL_G στις θέσεις των πολιτικών ευρωομάδων
df1999 = df1999.rename(columns={'SEATS_TOTAL_x':'SEATS_TOTAL_P'})
df1999 = df1999.rename(columns={'SEATS_TOTAL_y':'SEATS_TOTAL_G'})
df1999['YEAR'] = 1999
df1999 = pd.merge(df1999, eu1999, on='GROUP_ID', how='outer')
df1999

,PARTY_ID,SEATS_TOTAL_P,GROUP_ID,SEATS_TOTAL_G,SEATS_PERCENT_EU_x,LABEL,ACRONYM_y,YEAR,SEATS_TOTAL,SEATS_PERCENT_EU_y
0,EL01,9.0,GP0801,9.0,1.44,Ομάδα τoυ Ευρωπαϊκoύ Λαϊκoύ Κόμματoς (Χριστιαv...,N.D.,1999,232,37.06
1,EL02,9.0,GP0802,9.0,1.44,Ομάδα του Ευρωπαϊκού Σοσιαλιστικού Κόμματος,PA.SO.K.,1999,180,28.75
2,EL03,3.0,GP0805,7.0,1.12,Συνομοσπονδιακή Ομάδα της Ευρωπαϊκής Ενωτικής ...,KKE,1999,42,6.71
3,EL04,2.0,GP0805,7.0,1.12,Συνομοσπονδιακή Ομάδα της Ευρωπαϊκής Ενωτικής ...,SYN,1999,42,6.71
4,EL05,2.0,GP0805,7.0,1.12,Συνομοσπονδιακή Ομάδα της Ευρωπαϊκής Ενωτικής ...,DIKKI,1999,42,6.71
5,NaN,NaN,GP0803,0.0,0.00,Ομάδα του Ευρωπαϊκού Κόμματος των Φιλελευθέρων...,NaN,1999,50,7.99
6,NaN,NaN,GP0804,0.0,0.00,Ομάδα τωv Πρασίvωv / Ευρωπαϊκή Ελεύθερη Συμμαχία,NaN,1999,48,7.67
7,NaN,NaN,GP0806,0.0,0.00,Ομάδα ΄Εvωση για τηv Ευρώπη τωv Εθvώv,NaN,1999,31,4.95
8,NaN,NaN,GP0807,0.0,0.00,Ομάδα Τεχvικoύ Συvτovισμoύ τωv Αvεξαρτήτωv Βoυ...,NaN,1999,18,2.88
9,NaN,NaN,GP0808,0.0,0.00,Ομάδα για την Ευρώπη της Δημoκρατίας και της Δ...,NaN,1999,16,2.56
